In [11]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
import os

In [12]:
archivos = [
    "../data/clean/accidentalidad_limpia.csv",
    "../data/clean/ruta_linea154_limpia.csv",
    "../data/clean/estaciones_limpias.csv",
    "../data/clean/entradas_limpias.csv",
    "../data/clean/merge_limpio.csv"
]

for path in archivos:
    print(f"\nAnalizando: {path}")
    if not os.path.exists(path):
        print("❌ No existe.")
        continue
    try:
        df = pd.read_csv(path)
        if df.empty:
            print("⚠️ Archivo vacío.")
        elif df.columns.empty:
            print("⚠️ No hay columnas.")
        else:
            print(f"✔️ OK — {df.shape[0]} filas, {df.shape[1]} columnas.")
    except Exception as e:
        print("❌ Error al leer:", e)


Analizando: ../data/clean/accidentalidad_limpia.csv
✔️ OK — 50416 filas, 19 columnas.

Analizando: ../data/clean/ruta_linea154_limpia.csv
✔️ OK — 42 filas, 4 columnas.

Analizando: ../data/clean/estaciones_limpias.csv
✔️ OK — 801 filas, 10 columnas.

Analizando: ../data/clean/entradas_limpias.csv
❌ Error al leer: No columns to parse from file

Analizando: ../data/clean/merge_limpio.csv
✔️ OK — 50416 filas, 19 columnas.


In [2]:
try:
    print("Cargando Excel...")
    df_excel = pd.read_excel("../data/raw/AccidentalidadBicicletas_2024.xlsx", engine='openpyxl')
    df_excel.dropna(how="all", inplace=True)
    if "FECHA" in df_excel.columns:
        df_excel["FECHA"] = pd.to_datetime(df_excel["FECHA"], errors="coerce")
    print("Excel cargado correctamente.")
except Exception as e:
    print("Error al cargar Excel:", e)
    df_excel = pd.DataFrame()


Cargando Excel...
Excel cargado correctamente.


In [3]:
try:
    print("Cargando CSV...")
    df_csv = pd.read_csv("../data/raw/ruta_linea154.csv", encoding="utf-8")
    df_csv.dropna(how="all", inplace=True)
    print("CSV cargado correctamente.")
except Exception as e:
    print("Error al cargar CSV:", e)
    df_csv = pd.DataFrame()

Cargando CSV...
CSV cargado correctamente.


In [9]:
try:
    print("Cargando JSON...")
    with open("../data/raw/estacionesBicicletas.json", "r", encoding="utf-8") as f:
        data_json = json.load(f)

    # Soportar raíz como lista o como objeto con clave "estaciones"
    if isinstance(data_json, list):
        estaciones = data_json
    elif isinstance(data_json, dict) and "estaciones" in data_json:
        estaciones = data_json["estaciones"]
    else:
        estaciones = []

    if estaciones:
        df_json = pd.json_normalize(estaciones)
        df_json.dropna(how="all", inplace=True)
        df_json.to_csv("../data/clean/estaciones_limpias.csv", index=False)
        print("JSON cargado correctamente.")
    else:
        print("Advertencia: El JSON no contenía datos válidos.")
        df_json = pd.DataFrame()

except Exception as e:
    print("Error al cargar JSON:", e)
    df_json = pd.DataFrame()


Cargando JSON...
JSON cargado correctamente.


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

try:
    print("Cargando XML...")
    tree = ET.parse("../data/raw/2024Entradas.xml")
    root = tree.getroot()

    data_xml = []
    for elem in root.iter():
        if list(elem):  # Tiene hijos
            fila = {child.tag: child.text for child in elem}
            data_xml.append(fila)

    if data_xml:
        df_xml = pd.DataFrame(data_xml)
        df_xml.dropna(how="all", inplace=True)
        df_xml.to_csv("../data/clean/entradas_limpias.csv", index=False)
        print("XML limpio guardado.")
    else:
        print("Advertencia: el XML fue leído pero no se encontraron estructuras con datos.")
        df_xml = pd.DataFrame()
except Exception as e:
    print("Error al limpiar XML:", e)
    df_xml = pd.DataFrame()


Cargando XML...
Advertencia: El XML no contenía entradas válidas.


In [6]:
# === FUSIÓN / UNIÓN DE DATOS ===
try:
    print("Unificando datos por 'id' si corresponde...")
    if "id" in df_excel.columns and "id" in df_json.columns:
        df_merge = pd.merge(df_excel, df_json, on="id", how="inner")
    else:
        df_merge = pd.concat([df_excel, df_json], axis=0, ignore_index=True)
    print("Fusión completada.")
except Exception as e:
    print("Error durante la fusión:", e)
    df_merge = df_excel


Unificando datos por 'id' si corresponde...
Fusión completada.


In [7]:
# === GUARDAR ARCHIVOS LIMPIOS ===
print("Guardando archivos limpios en data/clean/")
df_excel.to_csv("../data/clean/accidentalidad_limpia.csv", index=False)
df_csv.to_csv("../data/clean/ruta_linea154_limpia.csv", index=False)
df_json.to_csv("../data/clean/estaciones_limpias.csv", index=False)
df_xml.to_csv("../data/clean/entradas_limpias.csv", index=False)
df_merge.to_csv("../data/clean/merge_limpio.csv", index=False)
print("Proceso de limpieza finalizado.")

Guardando archivos limpios en data/clean/
Proceso de limpieza finalizado.


In [14]:
# === Cargar los archivos limpios ===
print("Cargando archivos limpios desde data/clean/...")

df_excel = pd.read_csv("../data/clean/accidentalidad_limpia.csv")
df_csv = pd.read_csv("../data/clean/ruta_linea154_limpia.csv")
df_json = pd.read_csv("../data/clean/estaciones_limpias.csv")
df_xml = pd.read_csv("../data/clean/entradas_limpias.csv")
df_merge = pd.read_csv("../data/clean/merge_limpio.csv")

# === Visualización de las primeras filas ===
print("\n=== Accidentalidad (Excel limpio) ===")
print(df_excel.head())

print("\n=== Ruta Línea 154 (CSV limpio) ===")
print(df_csv.head())

print("\n=== Estaciones (JSON limpio) ===")
print(df_json.head())

print("\n=== Entradas (XML limpio) ===")
print(df_xml.head())

print("\n=== Datos fusionados (Merge) ===")
print(df_merge.head())


Cargando archivos limpios desde data/clean/...


EmptyDataError: No columns to parse from file